In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
test = pd.read_csv("/kaggle/input/playground-series-s3e22/test.csv")
train = pd.read_csv("/kaggle/input/playground-series-s3e22/train.csv")
train = train.drop(["id","hospital_number"],axis=1)
print(train.head())

In [ ]:
def make_contingency(column):
    contingency = pd.crosstab(train[column],train["outcome"])
    return contingency

In [ ]:
print(make_contingency("pain"))

In [ ]:
from scipy.stats import chi2_contingency
abcd={}
cramer = {}
for i in train.columns:
    arr= make_contingency(i)
#     arr= make_contingency(i).drop("euthanized",axis=1)
    arr+=.1
    chi2, p, dof, expected = chi2_contingency(arr)
    n = train.shape[0]
    k = arr.shape[1]
    r = arr.shape[0]
    
    cramerV= np.sqrt(chi2 / (n * min(k - 1, r - 1)))
    cramer[i]=cramerV

    abcd[i] = p
final = dict(sorted(abcd.items(), key=lambda x:x[1]))
CramerFinal = dict(sorted(cramer.items(), key=lambda x:x[1],reverse=True))

print(final)
print("\n" + str(CramerFinal))

In [ ]:
poss_col = []
poss_col2=[]
for i in list(final.keys()):
    if final[i]<=.05:
        poss_col.append(i)
for i in list(CramerFinal.keys()):
    if CramerFinal[i]>.4:
        poss_col2.append(i)
print(poss_col)
print(poss_col2)


In [ ]:
# train1 = train.drop(["lesion_3"],axis=1)
train1 = train
for i in train1.columns:
    if type(train1[i][0])==type("hello"):
        unique = list(train1[i].unique())
        for a in unique:
            train1[i]=train1[i].replace(a,unique.index(a)+1)

In [ ]:
def normalize_column(column):
    min_val = column.min()
    max_val = column.max()
    return (column - min_val) / (max_val - min_val)
train1 = train1.apply(normalize_column)
print(train1.head())

In [ ]:
datalist = (train1[poss_col2].values.tolist())
x_train = np.array([x[1:] for x in datalist])

y_train=np.zeros((len(x_train),3))
for i in range(len(datalist)):
    y_train[i][int(datalist[i][0]*2)]=1
x_train = x_train[:,np.newaxis,:]
print(x_train)

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
def sig(x):
  return 1/(1+np.exp(-x))
def sigDerv(x):
  return sig(x)*(1-sig(x))
def relu(x):
    return np.maximum(0,x)
def reluDerv(x):
    return np.where(x>0,1,0)

def cost(actual,predicted):
  return np.average((actual-predicted)**2)
def costDerv(actual,predicted):
  return 2*(predicted-actual)/predicted.size
class NeuralNetwork:
  def __init__(self):
    self.layers = []

  def add(self,layer):
    self.layers.append(layer)

  def train(self, input, actualOutput, trials):

    trialNumber = 0
    for x in range(trials): #Goes through each trial of practice
      success = 0
      total = 0
      for y in range(len(input)):
        output = input[y] #Technically first input (it's the data of the drawing)
        for z in self.layers: #Forward propagation
          output = z.forward(output) #Goes through the layers, getting an successful (?) prediction
        error = costDerv(actualOutput[y],output)
        if (actualOutput[y].argmax()==output.argmax()): #Success counter
          success+=1
        total+=1
        for z in reversed(self.layers): #Backward propagation
          error = z.backward(error)
      trialNumber+=1
      if(total==0):
        print("Invalid, division by zero| Successes " + str(success))
      else:
        print("Trial: "+ str(trialNumber)+"|"+ str(success) + "/" + str(total) + "|Percentage:" +str(round(success/total,3))) #Success calculator
  def test(self, input, actualOutput, trials):
    trialNumber = 0
    for x in range(trials): #Goes through each trial of practice
      success = 0
      total = 0
      for y in range(len(input)):
        output = input[y] #Technically first input (it's the data of the drawing)
        for z in self.layers: #Forward propagation
          output = z.forward(output) #Goes through the layers, getting an successful (?) prediction
        if (actualOutput[y].argmax()==output.argmax()): #Success counter
          success+=1
        total+=1
      trialNumber+=1
      if(total==0):
        print("Invalid, division by zero| Successes " + str(success))
      else:
        print("Trial: "+ str(trialNumber)+"|"+ str(success) + "/" + str(total) + "|Percentage:" +str(round(success/total,3))) #Success calculator
  def get_csv(self,data):
    
    results=[]
    for i in range(len(data)):
        output = data[i] #Technically first input (it's the data of the drawing)
        for z in self.layers: #Forward propagation
          output = z.forward(output) #Goes through the layers, getting an successful (?) prediction
        results.append(output[0].argmax())
    return results

class layer:
  def __init__(self,inputSize,outputSize): #i is input size, o is output size
    self.weights = np.random.rand(inputSize, outputSize) #Creates a i by o array of weights, the (0,2) represent connectetions between 3rd top neuron and next top neuron, etc
    self.bias = np.random.rand(1,outputSize) #Creates a 1 by o array of random biases [0,0.23,0.64,...]
  def forward(self,inputData):
    self.input = inputData
    self.output = sig(np.dot(self.input, self.weights) +self.bias) #self.output is an o by 1 array of outputs, activated by sig function
    return self.output
  def backward(self,outputError): #Output error is the gradient descent recursively given
    inputError = np.dot(outputError,self.weights.T)
    weightError = np.dot(self.input.T,outputError)
    self.weights -= weightError*.1
    self.bias -= outputError*.1
    return inputError*sigDerv(self.input) #returns the gradient descent deactivated of sig by chain rule


In [ ]:
net = NeuralNetwork()
net.add(layer(len(poss_col2)-1,64))
net.add(layer(64,3))
for a in range(30):
  randomX = np.random.randint(0,1135)
  print("Training " + str(a))
  net.train(x_train[randomX:randomX+100], y_train[randomX:randomX+100],1) #Testing the output
#   net.train(x_train, y_train,35) #training the network

# print("Network is now trained")
for a in range(100):
  randomX = np.random.randint(0,1135)
  print("Test " + str(a))
  net.test(x_train[randomX:randomX+100], y_train[randomX:randomX+100],1) #Testing the output 

In [ ]:
print(net.get_csv(x_train))
op=[]
for i in y_train:
    op.append(i.argmax())
print(op)

In [ ]:
for i in test1.columns:
    if type(test1[i][0])==type("hello"):
        unique = list(test1[i].unique())
        for a in unique:
            test1[i]=test1[i].replace(a,unique.index(a)+1)
test1 = test1.apply(normalize_column)
newTest = test1[poss_col2[1:]]


datalist = (newTest.values.tolist())
x_test = np.array([x for x in datalist])
x_test = x_test[:,np.newaxis,:]
ANSWER = net.get_csv(x_test)

print(ANSWER)

In [ ]:
import csv
with open('/kaggle/working/HenryWang.csv', "w") as file:
    writer = csv.writer(file)
    writer.writerow(["id","outcome"])
    for row in range(len(ANSWER)):
        if ANSWER[row]==2:
            outcome="lived"
        elif ANSWER[row]==1:
            outcome="euthanized"
        else:
            outcome="died"
        writer.writerow([test["id"][row],outcome])
    